In [1]:
!pip install pynvml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install numpy==1.16.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# !pip uninstall pycocotools
# !pip install pycocotools-windows

In [4]:
import sys
sys.path.append('/content/drive/MyDrive/CapastoneProject/codebase/')

In [5]:
import argparse
import os
import numpy as np
import time
import data
from importlib import import_module
import shutil
from utils import *
from split_combine import SplitComb

import torch
from torch.nn import DataParallel
from torch.backends import cudnn
from torch.utils.data import DataLoader
from torch import optim
from torch.autograd import Variable
# from config_training import config as config_training

from layers import acc
import gc

gc.collect()

torch.cuda.empty_cache()

/usr/local/lib/python3.7/dist-packages/scipy/__init__.py:149: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.16.1
  UserWarning)


In [6]:
def get_lr(epoch):
    if epoch <= args.epochs * 1/3: #0.5:
        lr = args.lr
    elif epoch <= args.epochs * 2/3: #0.8:
        lr = 0.1 * args.lr
    elif epoch <= args.epochs * 0.8:
        lr = 0.05 * args.lr
    else:
        lr = 0.01 * args.lr
    return lr


In [7]:
config_training = {'train_data_path':['/content/drive/MyDrive/CapastoneProject/data/luna16/subset0/'
                             ],
          'val_data_path':['/content/drive/MyDrive/CapastoneProject/data/luna16/subset1/'], 
          'test_data_path':['/content/drive/MyDrive/CapastoneProject/data/luna16/subset2/'], 
          
          'train_preprocess_result_path':'/content/drive/MyDrive/CapastoneProject/data/luna16/preprocess/',
          'val_preprocess_result_path':'/content/drive/MyDrive/CapastoneProject/data/luna16/preprocess/',  
          'test_preprocess_result_path':'/content/drive/MyDrive/CapastoneProject/data/luna16/preprocess/',
          
          'train_annos_path':'/content/drive/MyDrive/CapastoneProject/data/luna16/CSVFILES/annotations.csv',
          'val_annos_path':'/content/drive/MyDrive/CapastoneProject/data/luna16/CSVFILES/annotations.csv',
          'test_annos_path':'/content/drive/MyDrive/CapastoneProject/data/luna16/CSVFILES/annotations.csv',

          'black_list':[],
          
          'preprocessing_backend':'python',

          'luna_segment':'/content/drive/MyDrive/CapastoneProject/data/luna16/seg-lungs-LUNA16/',
          'preprocess_result_path':'/content/drive/MyDrive/CapastoneProject/data/luna16/preprocess/',
          'luna_data':'/content/drive/MyDrive/CapastoneProject/data/luna16/',
          'luna_label':'/content/drive/MyDrive/CapastoneProject/data/luna16/CSVFILES/annotations.csv'}

In [8]:
parser = argparse.ArgumentParser(description='PyTorch DataBowl3 Detector')
parser.add_argument('--model', '-m', metavar='MODEL', default='base',
                    help='model')
parser.add_argument('--config', '-c', default='config_training', type=str)
parser.add_argument('-j', '--workers', default=2, type=int, metavar='N',
                    help='number of data loading workers (default: 32)')
parser.add_argument('--epochs', default=100, type=int, metavar='N',
                    help='number of total epochs to run')
parser.add_argument('--start-epoch', default=0, type=int, metavar='N',
                    help='manual epoch number (useful on restarts)')
parser.add_argument('-b', '--batch-size', default=16, type=int,
                    metavar='N', help='mini-batch size (default: 16)')
parser.add_argument('--lr', '--learning-rate', default=0.01, type=float,
                    metavar='LR', help='initial learning rate')
parser.add_argument('--momentum', default=0.9, type=float, metavar='M',
                    help='momentum')
parser.add_argument('--weight-decay', '--wd', default=1e-4, type=float,
                    metavar='W', help='weight decay (default: 1e-4)')
parser.add_argument('--save-freq', default='1', type=int, metavar='S',
                    help='save frequency')
parser.add_argument('--resume', default='', type=str, metavar='PATH',
                    help='path to latest checkpoint (default: none)')
parser.add_argument('--save-dir', default='', type=str, metavar='SAVE',
                    help='directory to save checkpoint (default: none)')
parser.add_argument('--test', default=0, type=int, metavar='TEST',
                    help='1 do test evaluation, 0 not')
parser.add_argument('--testthresh', default=-3, type=float,
                    help='threshod for get pbb')
parser.add_argument('--split', default=8, type=int, metavar='SPLIT',
                    help='In the test phase, split the image to 8 parts')
parser.add_argument('--gpu', default='all', type=str, metavar='N',
                    help='use gpu')
parser.add_argument('--n_test', default=4, type=int, metavar='N',
                    help='number of gpu for test')


_StoreAction(option_strings=['--n_test'], dest='n_test', nargs=None, const=None, default=4, type=<class 'int'>, choices=None, help='number of gpu for test', metavar='N')

In [9]:
global args
args = parser.parse_args(['--model', 'res18', '-b', '16', '--resume', '064.ckpt', '--save-dir', 'res18/retrft96$f/', '--epochs', '5'])

In [10]:
def train(data_loader, net, loss, epoch, optimizer, get_lr, save_freq, save_dir):
    start_time = time.time()
    
    net.train()
    lr = get_lr(epoch)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    metrics = []

    for i, (data, target, coord) in enumerate(data_loader):
        data = Variable(data.cuda(non_blocking = True))
        target = Variable(target.cuda(non_blocking = True))
        coord = Variable(coord.cuda(non_blocking = True))

        output = net(data, coord)
        loss_output = loss(output, target)
        optimizer.zero_grad()
        loss_output[0].backward()
        optimizer.step()

        loss_output[0] = loss_output[0].data
        metrics.append(loss_output)

    if epoch % args.save_freq == 0:            
        state_dict = net.module.state_dict()
        for key in state_dict.keys():
            state_dict[key] = state_dict[key].cpu()
            
        torch.save({
            'epoch': epoch,
            'save_dir': save_dir,
            'state_dict': state_dict,
            'args': args},
            os.path.join(save_dir, '%03d.ckpt' % epoch))

    end_time = time.time()

    metrics = np.asarray(metrics, np.float32)
    print('Epoch %03d (lr %.5f)' % (epoch, lr))
    print('Train:      tpr %3.2f, tnr %3.2f, total pos %d, total neg %d, time %3.2f' % (
        100.0 * np.sum(metrics[:, 6]) / np.sum(metrics[:, 7]),
        100.0 * np.sum(metrics[:, 8]) / np.sum(metrics[:, 9]),
        np.sum(metrics[:, 7]),
        np.sum(metrics[:, 9]),
        end_time - start_time))
    print('loss %2.4f, classify loss %2.4f, regress loss %2.4f, %2.4f, %2.4f, %2.4f' % (
        np.mean(metrics[:, 0]),
        np.mean(metrics[:, 1]),
        np.mean(metrics[:, 2]),
        np.mean(metrics[:, 3]),
        np.mean(metrics[:, 4]),
        np.mean(metrics[:, 5])))
    print()

In [11]:
def validate(data_loader, net, loss):
    start_time = time.time()
    
    net.eval()

    metrics = []
    for i, (data, target, coord) in enumerate(data_loader):
        data = Variable(data.cuda(non_blocking = True), volatile = True)
        target = Variable(target.cuda(non_blocking = True), volatile = True)
        coord = Variable(coord.cuda(non_blocking = True), volatile = True)

        output = net(data, coord)
        loss_output = loss(output, target, train = False)

        loss_output[0] = loss_output[0].data[0]
        metrics.append(loss_output)    
    end_time = time.time()

    metrics = np.asarray(metrics, np.float32)
    print('Validation: tpr %3.2f, tnr %3.8f, total pos %d, total neg %d, time %3.2f' % (
        100.0 * np.sum(metrics[:, 6]) / np.sum(metrics[:, 7]),
        100.0 * np.sum(metrics[:, 8]) / np.sum(metrics[:, 9]),
        np.sum(metrics[:, 7]),
        np.sum(metrics[:, 9]),
        end_time - start_time))
    print('loss %2.4f, classify loss %2.4f, regress loss %2.4f, %2.4f, %2.4f, %2.4f' % (
        np.mean(metrics[:, 0]),
        np.mean(metrics[:, 1]),
        np.mean(metrics[:, 2]),
        np.mean(metrics[:, 3]),
        np.mean(metrics[:, 4]),
        np.mean(metrics[:, 5])))
    print()
    print()

In [12]:
def test(data_loader, net, get_pbb, save_dir, config):
    start_time = time.time()
    save_dir = os.path.join(save_dir,'bbox')
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    print(save_dir)
    net.eval()
    namelist = []
    split_comber = data_loader.dataset.split_comber
    for i_name, (data, target, coord, nzhw) in enumerate(data_loader):
        s = time.time()
        target = [np.asarray(t, np.float32) for t in target]
        lbb = target[0]
        nzhw = nzhw[0]
        name = data_loader.dataset.filenames[i_name].split('/')[-1].split('_clean')[0] #.split('-')[0]  wentao change
        data = data[0][0]
        coord = coord[0][0]
        isfeat = False
        if 'output_feature' in config:
            if config['output_feature']:
                isfeat = True
        n_per_run = args.n_test
        print(data.size())
        splitlist = range(0,len(data)+1,n_per_run)
        if splitlist[-1]!=len(data):
            splitlist.append(len(data))
        outputlist = []
        featurelist = []

        for i in range(len(splitlist)-1):
            input = Variable(data[splitlist[i]:splitlist[i+1]], volatile = True).cuda()
            inputcoord = Variable(coord[splitlist[i]:splitlist[i+1]], volatile = True).cuda()
            if isfeat:
                output,feature = net(input,inputcoord)
                featurelist.append(feature.data.cpu().numpy())
            else:
                output = net(input,inputcoord)
            outputlist.append(output.data.cpu().numpy())
        output = np.concatenate(outputlist,0)
        output = split_comber.combine(output,nzhw=nzhw)
        if isfeat:
            feature = np.concatenate(featurelist,0).transpose([0,2,3,4,1])[:,:,:,:,:,np.newaxis]
            feature = split_comber.combine(feature,sidelen)[...,0]

        thresh = args.testthresh # -8 #-3
        print('pbb thresh', thresh)
        pbb,mask = get_pbb(output,thresh,ismask=True)
        if isfeat:
            feature_selected = feature[mask[0],mask[1],mask[2]]
            np.save(os.path.join(save_dir, name+'_feature.npy'), feature_selected)
        #tp,fp,fn,_ = acc(pbb,lbb,0,0.1,0.1)
        #print([len(tp),len(fp),len(fn)])
        print([i_name,name])
        e = time.time()
        np.save(os.path.join(save_dir, name+'_pbb.npy'), pbb)
        np.save(os.path.join(save_dir, name+'_lbb.npy'), lbb)
    np.save(os.path.join(save_dir, 'namelist.npy'), namelist)
    end_time = time.time()

    print('elapsed time is %3.2f seconds' % (end_time - start_time))
    print()
    print()

In [13]:
def singletest(data,net,config,splitfun,combinefun,n_per_run,margin = 64,isfeat=False):
    z, h, w = data.size(2), data.size(3), data.size(4)
    print(data.size())
    data = splitfun(data,config['max_stride'],margin)
    data = Variable(data.cuda(non_blocking = True), volatile = True,requires_grad=False)
    splitlist = range(0,args.split+1,n_per_run)
    outputlist = []
    featurelist = []
    for i in range(len(splitlist)-1):
        if isfeat:
            output,feature = net(data[splitlist[i]:splitlist[i+1]])
            featurelist.append(feature)
        else:
            output = net(data[splitlist[i]:splitlist[i+1]])
        output = output.data.cpu().numpy()
        outputlist.append(output)
        
    output = np.concatenate(outputlist,0)
    output = combinefun(output, z / config['stride'], h / config['stride'], w / config['stride'])
    if isfeat:
        feature = np.concatenate(featurelist,0).transpose([0,2,3,4,1])
        feature = combinefun(feature, z / config['stride'], h / config['stride'], w / config['stride'])
        return output,feature
    else:
        return output

In [14]:
torch.manual_seed(0)
torch.cuda.set_device(0)

model = import_module(args.model)
config, net, loss, get_pbb = model.get_model()
start_epoch = args.start_epoch
save_dir = args.save_dir

In [15]:
# if args.resume:
#         checkpoint = torch.load(args.resume)
#         # if start_epoch == 0:
#         #     start_epoch = checkpoint['epoch'] + 1
#         # if not save_dir:
#         #     save_dir = checkpoint['save_dir']
#         # else:
#         #     save_dir = os.path.join('results',save_dir)
#         net.load_state_dict(checkpoint['state_dict'])
#     # else:

In [16]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
t,r,a,f

(15843721216, 0, 0, 0)

In [17]:
if start_epoch == 0:
    start_epoch = 1
if not save_dir:
    exp_id = time.strftime('%Y%m%d-%H%M%S', time.localtime())
    save_dir = os.path.join('results', args.model + '-' + exp_id)
else:
    save_dir = os.path.join('results',save_dir)

if not os.path.exists(save_dir):
    os.makedirs(save_dir)
logfile = os.path.join(save_dir,'log')
if args.test!=1:
    sys.stdout = Logger(logfile)
    pyfiles = [f for f in os.listdir('./') if f.endswith('.py')]
    for f in pyfiles:
        shutil.copy(f,os.path.join(save_dir,f))

In [18]:
n_gpu = setgpu(args.gpu)
args.n_gpu = n_gpu
net = net.cuda()
loss = loss.cuda()
cudnn.benchmark = False #True
net = DataParallel(net)
traindatadir = config_training['train_preprocess_result_path']
valdatadir = config_training['val_preprocess_result_path']
testdatadir = config_training['test_preprocess_result_path']
trainfilelist = []
print(config_training['train_data_path'])
for folder in config_training['train_data_path']:
    print(folder)
    for f in os.listdir(folder):
        if f.endswith('.mhd') and f[:-4] not in config_training['black_list']:
            trainfilelist.append(folder.split('/')[-2]+'/'+f[:-4])
valfilelist = []
for folder in config_training['val_data_path']:
    for f in os.listdir(folder):
        if f.endswith('.mhd') and f[:-4] not in config_training['black_list']:
            valfilelist.append(folder.split('/')[-2]+'/'+f[:-4])
testfilelist = []
for folder in config_training['test_data_path']:
    for f in os.listdir(folder):
        if f.endswith('.mhd') and f[:-4] not in config_training['black_list']:
            testfilelist.append(folder.split('/')[-2]+'/'+f[:-4])

In [19]:
if args.test == 1:
    margin = 32
    sidelen = 144
    import data
    split_comber = SplitComb(sidelen,config['max_stride'],config['stride'],margin,config['pad_value'])
    dataset = data.DataBowl3Detector(
        testdatadir,
        testfilelist,
        config,
        phase='test',
        split_comber=split_comber)
    test_loader = DataLoader(
        dataset,
        batch_size = 1,
        shuffle = False,
        num_workers = args.workers,
        collate_fn = data.collate,
        pin_memory=False)

    for i, (data, target, coord, nzhw) in enumerate(test_loader): # check data consistency
        if i >= len(testfilelist)/args.batch_size:
            break
    
    test(test_loader, net, get_pbb, save_dir,config)



In [20]:
args

Namespace(batch_size=16, config='config_training', epochs=5, gpu='all', lr=0.01, model='res18', momentum=0.9, n_gpu=1, n_test=4, resume='064.ckpt', save_dir='res18/retrft96$f/', save_freq=1, split=8, start_epoch=0, test=0, testthresh=-3, weight_decay=0.0001, workers=2)

In [21]:
config

{'anchors': [5.0, 10.0, 20.0],
 'aug_scale': True,
 'augtype': {'flip': True, 'rotate': False, 'scale': True, 'swap': False},
 'blacklist': ['868b024d9fa388b7ddab12ec1c06af38',
  '990fbe3f0a1b53878669967b9afd1441',
  'adc3bbc63d40f8761c59be10f1e504c3'],
 'bound_size': 12,
 'chanel': 1,
 'crop_size': [96, 96, 96],
 'max_stride': 16,
 'num_hard': 2,
 'num_neg': 800,
 'pad_value': 170,
 'r_rand_crop': 0.3,
 'reso': 1,
 'sizelim': 2.5,
 'sizelim2': 10,
 'sizelim3': 20,
 'stride': 4,
 'th_neg': 0.02,
 'th_pos_train': 0.5,
 'th_pos_val': 1}

In [22]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
t,r,a,f

(15843721216, 33554432, 21625856, 11928576)

In [23]:
import data
print(len(trainfilelist))
dataset = data.DataBowl3Detector(
    traindatadir,
    trainfilelist,
    config,
    phase = 'train')

train_loader = DataLoader(
    dataset,
    batch_size = args.batch_size,
    shuffle = True,
    num_workers = args.workers,
    pin_memory=True)

# dataset = data.DataBowl3Detector(
#     valdatadir,
#     valfilelist,
#     config,
#     phase = 'val')
# val_loader = DataLoader(
#     dataset,
#     batch_size = args.batch_size,
#     shuffle = False,
#     num_workers = args.workers,
#     pin_memory=True)



In [24]:
# for i, (data, target, coord) in enumerate(train_loader): # check data consistency
#     if i >= int(len(trainfilelist)/args.batch_size):
#         break

# for i, (data, target, coord) in enumerate(val_loader): # check data consistency
#     if i >= len(valfilelist)/args.batch_size:
#         break

optimizer = torch.optim.SGD(
    net.parameters(),
    args.lr,
    momentum = 0.9,
    weight_decay = args.weight_decay)

for epoch in range(start_epoch, args.epochs + 1):
    print(epoch)
    train(train_loader, net, loss, epoch, optimizer, get_lr, args.save_freq, save_dir)
    # validate(val_loader, net, loss)

In [25]:
torch.cuda.empty_cache()